# Find the perpendicular distance from a point to each isochrone of a given age and also direct distance from Gaia parameters


In [1]:
import pickle
import os
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
os.chdir('Data')
pkl_file = open('isochrones.pkl', 'rb')
stacked_isochrones = pickle.load(pkl_file)

pkl_file = open('columns.pkl', 'rb')
x_columns = pickle.load(pkl_file)

pkl_file = open('x_values.pkl', 'rb')
x_values = pickle.load(pkl_file)

pkl_file = open('isoc_cols.pkl', 'rb')
isoc_columns = pickle.load(pkl_file)

x_input=pd.read_csv('x_input')
x_input_err=pd.read_csv('x_input_err')

os.chdir('..')

In [2]:
import torch
def find_nearest(array, value):
    #array = np.asarray(array)
    idx = (torch.abs(array - value)).argmin()
    return array[idx],idx

def isochrone_selector(feh,age):
    '''if(feh<-4 or feh>0.5):
        raise NotImplementedError
    if(age<5 or age>10.3):
        raise NotImplementedError
    else:'''
    logagegrid = torch.tensor(np.linspace(5,10.3,105))
    fehgrid = torch.tensor(np.linspace(-4,0.5,90))
    feh,feh_idx=find_nearest(fehgrid,feh)
    age,age_idx=find_nearest(logagegrid,age)

    return feh_idx*len(logagegrid)+age_idx

def column_index(name):
    if name in x_columns:
        return np.where(np.array(x_columns)==name)[0][0]
    else:
        return np.where(np.array(isoc_columns)==name)[0][0] +len(x_columns)

In [10]:
stacked_isochrones.shape

torch.Size([10, 15, 435])

# Find the perpendicular values

In [4]:
stacked_isochrones=torch.tensor(stacked_isochrones)

In [5]:
x_values=torch.tensor(x_values)

In [6]:
stacked_isochrones.shape

torch.Size([10, 15, 435])

In [7]:
def perpendicular_dist(i,age):

    isochrone=torch.cat([x_values[i],stacked_isochrones[isochrone_selector(x_values[i][column_index('mh_gspphot') ][0],age)]],dim=0)
    isochrone=isochrone.reshape((1,)+isochrone.shape)


    truth_1=(isochrone[:,column_index('G'),:]+-(isochrone[:,column_index('bp_rp'),:])*isochrone[:,column_index('p_slopes'),:]<=isochrone[:,column_index('high_c'),:]) #box selection
    truth_1=truth_1.reshape(truth_1.shape[0],1,truth_1.shape[1])
    truth_2=(isochrone[:,column_index('low_c'),:]<=isochrone[:,column_index('G'),:]-(isochrone[:,column_index('bp_rp'),:])*isochrone[:,column_index('p_slopes'),:])
    truth_2=truth_2.reshape(truth_2.shape[0],1,truth_2.shape[1])
    truth=truth_1*truth_2
    # ^box selection

    # projection onto the nearest line
    x=((1/torch.sqrt(1+isochrone[:,column_index('slopes'),:]**2))*(isochrone[:,column_index('G'),:]-(isochrone[:,column_index('bp_rp'),:])*isochrone[:,column_index('slopes'),:]-isochrone[:,column_index('Gaia_G_EDR3'),:] + isochrone[:,column_index('slopes'),:]*isochrone[:,column_index('BPRP'),:]))
    # taking the minimum
    idx=torch.argmin(torch.abs(x/truth.reshape(x.shape)).nan_to_num(nan=torch.inf),1)
    x=x.gather(1,idx.view(-1,1))
    #error needs to be corrected for absolute magnitude 
    x_err=(1/(1+isochrone[:,column_index('slopes'),:]**2))*isochrone[:,column_index('phot_g_mean_mag_error'),:]**2+(isochrone[:,column_index('slopes'),:]*isochrone[:,column_index('bp_rp_error'),:])**2
    x_err=x_err.gather(1,idx.view(-1,1))
    isochrone=torch.cat((isochrone,x.reshape(x.shape[0],1,x.shape[1]).repeat(1,1,(isochrone).shape[-1]),x_err.reshape(x_err.shape[0],1,x_err.shape[1]).repeat(1,1,(isochrone).shape[-1])),1)

    dist=torch.distributions.multivariate_normal.MultivariateNormal(torch.zeros_like(x),torch.eye(len(x))+torch.diag(x_err**2))
    
    try:# serious issues here
                
        log_l=dist.log_prob(x)
                

    except:
                
        log_l=0.0

    return log_l, isochrone            

In [8]:
perpendicular_dist(1,6)

IndexError: index 6845 is out of bounds for dimension 0 with size 10